In [16]:
# Global Variables set by user
PROJECT_PORTAL = "https://envisioning.maps.arcgis.com"
PROJECT_USERNAME = "GBushongENV"
PROJECT_GROUPS = ["aec-test-group", "aec-test-group-2"] # specify groups to be shared with delivery org
DELIVERY_PORTAL = "https://esrienergy.maps.arcgis.com"
DELIVERY_USERNAME = "PDOAdmin"
COPY_PREFIX = "Test" # the word or words that should be appended to the front to distinguish these copies of the template items
FOLDER_SUFFIX = "Content" # the folder containing these copies of files will be named COPY_PREFIX + FOLDER_SUFFIX

In [17]:
# import libraries
from arcgis.gis import GIS
from IPython.display import display
import tempfile

In [18]:
# connections to Project and Delivery organizations
project = GIS(PROJECT_PORTAL, PROJECT_USERNAME)
delivery_org = GIS(DELIVERY_PORTAL, DELIVERY_USERNAME)
GROUP_NAME_STRUCTURE = COPY_PREFIX + "-{}"
NAME_STRUCTURE = COPY_PREFIX + " {}"
#FOLDER = COPY_PREFIX + " " + FOLDER_SUFFIX
FOLDER = NAME_STRUCTURE.format(FOLDER_SUFFIX)

Enter password: ········
Enter password: ········


In [21]:
# functions from utils/clone_utils
def search_by_title(target, title, obj_type):
    """search org for an existing item with title
    args:
    target -- target GIS to search
    title -- item or group to search
    obj_type -- can be "item" or "group"
    """
    if(obj_type == "group"):
        s_items = target.groups.search(title)
    elif(obj_type == "item"):
        s_items = target.content.search(query='title:{}'.format(title))
    for s_item in s_items:
        if s_item.title == title:
            return s_item
    return None

# functions to modify the title of the cloned items and groups
def modify_item(item, target):
    title = NAME_STRUCTURE.format(item.title)
    while search_by_title(target, title, "item"):
        title = input("Title `{0}` for ITEM `{1}` already exists \nNew title: ".format(title, item.title))
    return {"title": title}

def modify_group(expected_title, target):
    title = GROUP_NAME_STRUCTURE.format(expected_title)
    while search_by_title(target, title, "group"):
        title = input("Title `{0}` for GROUP `{1}` already exists \nNew title: ".format(title, expected_title))
    return {"title": title}

def clone_group_modify(group, target):
    res = modify_group(group.title, target)
    new_group_title = res['title']
    if not search_by_title(project, new_group_title, "group"):
        try:
            with tempfile.TemporaryDirectory() as temp_dir:
                thumbnail_file = group.download_thumbnail(temp_dir)
                target_group = project.groups.create(title=new_group_title, tags=group.tags,
                                                    description=group.description, snippet=group.snippet,
                                                    access=group.access, thumbnail=thumbnail_file,
                                                    is_invitation_only=True, sort_field='avgRating',
                                                    sort_order='asc', is_view_only=False)
                display(target_group)
            return target_group
        except Exception as e:
            print('Group {} could not be created: {}'.format(new_group_title, e))
            return None
    else: # shouldn't ever get here because modify_group should ensure a unique group name
        print('Group {} already exists in portal'.format(new_group_title))
        return None
        
def clone_items_modify(group, target, modify_item_callback=None, modify_group_callback=None, **kwargs):
    """Clone groups and items to a target GIS
    * Abstraction over arcgis.gis.ContentManager.clone_items to:
    - Provide callbacks to update properties, sometimes shouldn't have 1:1 copy

    args:
    group -- group containing items to be cloned. cloned group name based off 'GROUP_NAME_STRUCTURE'
    target -- target gis where items or groups will be cloned
    modify_item_callback -- callback to update Item properties, expects args: (item_clone, target_gis)
        returns: flattened dict of args, eg {'title':'<>'; 'data':'<>'}, args here:
    https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html?highlight=clone_items#arcgis.gis.Item.update

    modify_group_callback -- callback to update Group properties, expects args: (expected_title, target_gis)
        returns: dict of args here:
    https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html?highlight=clone_items#arcgis.gis.Group.update

    **kwargs:
    all additional args go into the gis.ContentManager.clone_items function described here:
    https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html?highlight=clone_items#arcgis.gis.ContentManager.clone_items
    """

    from arcgis.gis import Group
    from arcgis.gis import Item

    # allow single values
    items = group if isinstance(group, list) else [group]
    
    # clone the group
    new_group = clone_group_modify(group, target)
    group_map = {group.id: new_group.id} # put cloned items in the new group just created
    
    # clone items in the specified group to the folder
    results = target.content.clone_items(items, group_mapping=group_map, **kwargs) 

    # update each result
    for result in results:
        if isinstance(result, Item) and modify_item_callback:
            props = modify_item_callback(result, target)
            args = {}
            args['data'] = props.pop('data', None)
            args['thumbnail'] = props.pop('thumbnail', None)
            args['metadata'] = props.pop('metadata', None)
            args['item_properties'] = props
            result.update(**args)
        else:
            print(result)
            
    return results

In [22]:
# get and print project groups to copy to all new delivery orgs
project_groups = [search_by_title(project, title, "group") for title in PROJECT_GROUPS]

for group in project_groups:
    display(group)

<Group title:"aec-test-group" owner:GBushongENV>

<Group title:"aec-test-group-2" owner:GBushongENV>

In [23]:
# copy project groups to project org (external version)
for group in project_groups:
    clone_res = clone_items_modify(group, project, modify_item_callback=modify_item, 
                                   modify_group_callback=modify_group, 
                                   copy_data=False, search_existing_items=False, folder=FOLDER)
    display(clone_res)

Title `Test-aec-test-group` for GROUP `aec-test-group` already exists 
New title: TEST-1


<Group title:"TEST-1" owner:GBushongENV>

Title `Test Sample Feature Layer` for ITEM `Sample Feature Layer` already exists 
New title: SFL 1
Title `Test Sample Map` for ITEM `Sample Map` already exists 
New title: SM 1
Title `Test Sample Scene` for ITEM `Sample Scene` already exists 
New title: SS 1


[<Item title:"SFL 1" type:Feature Layer Collection owner:GBushongENV>,
 <Item title:"SM 1" type:Web Map owner:GBushongENV>,
 <Item title:"SS 1" type:Web Scene owner:GBushongENV>]

Title `Test-aec-test-group-2` for GROUP `aec-test-group-2` already exists 
New title: TEST-2


<Group title:"TEST-2" owner:GBushongENV>

Title `Test Sample Locator` for ITEM `Sample Locator` already exists 
New title: SL 1


[<Item title:"SL 1" type:Geocoding Layer owner:GBushongENV>]